In [4]:
mkdir ./data_testing

In [16]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains # <--- helpful!
from selenium.webdriver.common.by import By
import configparser

from bs4 import BeautifulSoup

import pandas as pd
from pandas.io.json import json_normalize

import os
from time import sleep
#import tqdm
import sys

## Instagram Comment Getter

Running in jhub because the docker version wasn't getting all comments for some reason...

In [17]:
# Our instagram df from PhantomBuster
ig_df = pd.read_csv('./data/blackivystories_PostsExtractor_03012021.csv')

# Our exported Azure OCR df with text
ocr_df = pd.read_json('./data/blackivystories_ig_ocr_expanded.json')

ivy_input_name = 'penn'

In [18]:
postid_filename = []
penn_ocr_pid = []

for pid in ocr_df[ocr_df.ivy == ivy_input_name].filename.str[:-4]:
    penn_ocr_pid.append(int(pid))
    
pennstory_ig_post_urls = ig_df[ig_df.postId.isin(penn_ocr_pid)].postUrl.unique()


In [19]:
len(pennstory_ig_post_urls)

55

## Prepare the out dirs:

In [29]:
mkdir ./data/raw

mkdir: ./data/raw: File exists


In [30]:
mkdir ./data/etl

mkdir: ./data/etl: File exists


### Docker Selenium Container Browser

Confirmed this is working in firefox, now switching to docker again...

Make sure to set your `driver` in the cell below:

______

``` python
# DEV / TESTING
# driver = webdriver.Firefox() 

# DEPLOYMENT
selenium_url = 'http://localhost:4444/wd/hub' # <-- docker standalonee selenium chrome browser
options.add_argument('--headless')
driver = webdriver.Remote(command_executor=selenium_url,desired_capabilities=options.to_capabilities())
```
_______

I think my bash deployment script was off, for this many URLs it isn't terrible to run in a notebook on a headless container browser

### *RERUN THIS CELL TO START WHERE YOU LEFT OFF IF BELOW FAILS...*

In [96]:
# get our ids
penn_url_ids = []

for url in pennstory_ig_post_urls:
    penn_url_ids.append(url.split('/')[-2])
    
# check if they already ran (in case the cell fails)
url_for_commentgetter = []

for ig_url in penn_url_ids:
    json_outfile = ig_url + '.json'
    if json_outfile not in os.listdir('./data/etl/'): # <-- the url doesn't already have a json etl outfile
        url_for_commentgetter.append('https://www.instagram.com/p/{}/'.format(ig_url))

In [97]:
!docker stop selenium-chrome

selenium-chrome


### Let us try the firefox standalone docker instead of chrome ... 

In [98]:
!docker run -d -p 4444:4444 --shm-size 2g selenium/standalone-firefox

42702f29043b850010c4406922bf2da076ec9fc8d3df84801cd57b641fc05d41
docker: Error response from daemon: driver failed programming external connectivity on endpoint nostalgic_ardinghelli (e914a0f37b299b81f6ce28ea3f0e011185da1931e5480f1dbb08ecbf74ff4319): Bind for 0.0.0.0:4444 failed: port is already allocated.


In [104]:
!docker ps

CONTAINER ID   IMAGE                         COMMAND                  CREATED         STATUS         PORTS                    NAMES
35faafb3499e   selenium/standalone-firefox   "/opt/bin/entry_poin…"   2 minutes ago   Up 2 minutes   0.0.0.0:4444->4444/tcp   selenium-firefox


###  Loop through 55 URLs to scrape comments

Wait a few seconds for the docker `selenium-chrome` container to restart...

Since these posts do not have a lot of comments it likely would work to lower opt param for failed threshold click from 10 to like 4... 

In [105]:
pennstory_ig_post_urls[0:4]

array(['https://www.instagram.com/p/CG-Q1JuhVAB/',
       'https://www.instagram.com/p/CGsyLsUhk0K/',
       'https://www.instagram.com/p/CGioTquByeG/',
       'https://www.instagram.com/p/CE4hEsjhkWP/'], dtype=object)

In [106]:
url_for_commentgetter[0:4]

['https://www.instagram.com/p/CDuAQx-h5TZ/',
 'https://www.instagram.com/p/CDrxRxghJPy/',
 'https://www.instagram.com/p/CDrVs6nhpYN/',
 'https://www.instagram.com/p/CDpS3mvhCjA/']

In [107]:
len(url_for_commentgetter)

45

In [109]:
for url in url_for_commentgetter:
    
    url = url[:-1] # <-- remove trailing slash
    print(url)
    print('-'*60)

    # Using one example image
    #url = 'https://www.instagram.com/p/CDFqYo7BxDv'# <-- no trailing slash on url
    #url = sys.argv[1] # <-- Second argument of the dockerfile CMD (first argument is the python script name itself!)
    
    if not url.startswith('https://'):
        url = 'https://' + url

    # Raw HTML file outdir
    raw_outfile = "./data/raw/{}.html".format(url.split('/')[-1]) # <--- outdir html file
    print('Raw html outfile prepared as: {}'.format(raw_outfile))

    # ETL (extracted from raw HTML) file outdir 
    etl_outfile = "./data/etl/{}.json".format(url.split('/')[-1]) # <--- outdir json file
    print('ETL outfile prepared as: {}'.format(etl_outfile))

    # Instagram HTML element
    xpath_aria_label_button = 'Load more comments' # <--- inspect & find class name for button click

    # Optimization parameters
    threshold_for_failed_click = 10  # <-- Int
    time_to_wait_for_nxt_click = 0.25 # <-- Int

    ##############################################
    # Setup Selenium Driver

    # driver = webdriver.Chrome(executable_path='/Users/etiennejacquot/Downloads/chromedriver_v84') 
    # driver = webdriver.Firefox()  # <-- desktop app firefox

    # options = webdriver.ChromeOptions() # <-- for selenium-chrome
    options = webdriver.FirefoxOptions() # for selnium-firefox
    selenium_url = 'http://localhost:4444/wd/hub' # <-- docker standalonee selenium browser
    options.add_argument('--headless')
    driver = webdriver.Remote(command_executor=selenium_url,desired_capabilities=options.to_capabilities())

    # Navigate to instagram post url
    driver.get(url) 

    i = 0 # <--- USED FOR CLICK COUNT
    j = 0 # <--- THRESHOLD FOR FAILED CLICKED 

    while j < threshold_for_failed_click:
        #print('J value is --> {}'.format(j))
        ##############################################
        # Sleep and give browser a chance to refresh
        sleep(time_to_wait_for_nxt_click) # <--- Time to wait until next click

        ##############################################
        # Try to find button to load more comments
        try: 
            element = driver.find_element(By.XPATH, '//button[./span[@aria-label="{}"]]'.format(xpath_aria_label_button))
        except:
            print("\nUnexpected error:", sys.exc_info()[0])
            j = j+1 # <--- Plus one for failed click
            continue

        ##############################################
        # Try to click button to load more comments
        print('Trying to click & load more comments --> {}'.format(i))

        try:
            ActionChains(driver).move_to_element(element).click().perform()
        except:
            print("\nUnexpected error:", sys.exc_info()[0])
            j=j+1 # <--- Plus one for failed click
            continue

        i=i+1 # <--- Successful cycle for clicked account, +1
        j = 0 # <--- FAILED CLICK COUNTER RESET

        if i > 500:
            print('oops! clicked 500 times apparently... not sure why this is happening so exiting this container to continue onward!')
            j=threshold_for_failed_click+1


    ##############################################
    # Exporting HTML raw data

    with open (raw_outfile,"w+") as f:
        print('Exporting to .html file --> {}'.format(raw_outfile))
        f.write(driver.page_source)

    ##############################################
    # ETL conversion
    driver.quit() # close out of chrome tab (or browser?)

    comment_df = pd.DataFrame()

    with open(raw_outfile, "r") as f:
        contents = f.read()
        soup = BeautifulSoup(contents, 'lxml')    

    # Comments are ul objects in HTML
    comments_html = soup.find_all('ul')

    # Loop through comments in html file to extract ETL
    for x in comments_html:

        a = x.find('div') 

        # get author name
        try:
            author = a.get_text(separator=' ').split(' ')[0] # <--- No spaces in Instagram usernames!
        except:
            author = None
            pass 

        # get comment on post
        try:
            comment = a.get_text(separator=' ').split(' ')[1:]   
            comment = " ".join(comment) 
        except:
            comment = None
            pass

        comment_df = comment_df.append(pd.DataFrame({'url':[url],'author':[author],'comment':[comment]}))

    print('Done! Number of comments extracted --> {}'.format(comment_df.shape[0]))
    print('Exporting ETL json file --> {}'.format(etl_outfile))

    comment_df.reset_index(drop=True).to_json('{}'.format(etl_outfile)) 

    print('Complete!')
    print('-'*75)


https://www.instagram.com/p/CDuAQx-h5TZ
------------------------------------------------------------
Raw html outfile prepared as: ./data/raw/CDuAQx-h5TZ.html
ETL outfile prepared as: ./data/etl/CDuAQx-h5TZ.json

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'


Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
Exporting to .html file --> ./data/raw/CDkTx57BXHC.html
Done! Number of comments extracted --> 7
Exporting ETL json file --> ./data/etl/CDkTx57BXHC.json
Complete!
---------------------------------------------------------------------------
https://www.instagram.com/p/CDkLXzmB9Ao
------------------------------------------------------------
Raw html outfile prepared as: ./data/raw/CDkLXzmB9Ao.html
ETL outfile prepared as: ./data/etl/CDkLXzmB9Ao.json
Trying to click & load more comments --> 0

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selen


Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
Exporting to .html file --> ./data/raw/CDe-3p4BxLn.html
Done! Number of comments extracted --> 6
Exporting ETL json file --> ./data/etl/CDe-3p4BxLn.json
Complete!
---------------------------------------------------------------------------
https://www.instagram.com/p/CDVB4b7hgBl
----------

Trying to click & load more comments --> 1

Unexpected error: <class 'selenium.common.exceptions.MoveTargetOutOfBoundsException'>
Trying to click & load more comments --> 1

Unexpected error: <class 'selenium.common.exceptions.MoveTargetOutOfBoundsException'>
Trying to click & load more comments --> 1

Unexpected error: <class 'selenium.common.exceptions.MoveTargetOutOfBoundsException'>
Trying to click & load more comments --> 1

Unexpected error: <class 'selenium.common.exceptions.MoveTargetOutOfBoundsException'>
Trying to click & load more comments --> 1

Unexpected error: <class 'selenium.common.exceptions.MoveTargetOutOfBoundsException'>
Exporting to .html file --> ./data/raw/CDFqYo7BxDv.html
Done! Number of comments extracted --> 29
Exporting ETL json file --> ./data/etl/CDFqYo7BxDv.json
Complete!
---------------------------------------------------------------------------
https://www.instagram.com/p/CDAd29NBaZM
------------------------------------------------------------
Raw html 


Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
Exporting to .html file --> ./data/raw/CCtegCQBdJx.html
Done! Number of comments extracted --> 26
Exporting ETL json file --> ./data/etl/CCtegCQBdJx.json
Complete!
---------------------------------------------------------------------------
https://www.instagram.com/p/CCpDDT6B5nq
------------------------------------------------------------
Raw html outfile prepared as: ./data/raw/CCpDDT6B5nq.html
ETL outfile prepared as: ./data/etl/CCpDDT6B5nq.json
Trying to click & load more comments --> 0

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'sele


Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
Exporting to .html file --> ./data/raw/CCM5tgGhlUT.html
Done! Number of comments extracted --> 20
Exporting ETL json file --> ./data/etl/CCM5tgGhlUT.json
Complete!
---------------------------------------------------------------------------
https://www.instagram.com/p/CCKOBxZBubG
---------


Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>
Exporting to .html file --> ./data/raw/CB6UyVaD8Ez.html
Done! Number of comments extracted --> 13
Exporting ETL json file --> ./data/etl/CB6UyVaD8Ez.json
Complete!
---------------------------------------------------------------------------
https://www.instagram.com/p/CB1vWGmJJWc
------------------------------------------------------------
Raw html outfile prepared as: ./data/raw/CB1vWGmJJWc.html
ETL outfile prepared as: ./data/etl/CB1vWGmJJWc.json
Trying to click & load more comments --> 0

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'selenium.common.exceptions.NoSuchElementException'>

Unexpected error: <class 'sele

## Load ETL Json files to df

In [110]:
etl_files = [item for item in os.listdir('./data/etl/') if item.endswith('.json')]

dfs = []

for etl in etl_files:
    dfs.append(pd.read_json('./data/etl/{}'.format(etl)))
    
etl_df = pd.concat(dfs)

### 843 comments scrapped for 55 penn stories

In [114]:
etl_df.shape

(843, 3)

In [118]:
len(etl_df.url.unique())

55

## Top 20 most frequent authors of comments:

- The top two values are the account & view is just from the HTML and can be ignored

- the uofpenn I am unsure if it's actual university responses...

In [124]:
etl_df.author.value_counts().head(20)

blackivystories        55
View                   55
kathkvelle             30
uofpenn                24
jay_theorist           16
gelila.skm             16
kamiyerunsewe          10
czcharles              10
aquaquaintrelle         9
astrid.elisabet         9
aczhang                 9
gabriela.p.alvarado     9
gunveeng                9
emmaisidora             9
ataehcos._.kuo          8
megan.h.li              8
bettycfalcon            8
roxpog                  8
feyidapharaoh           8
axj_spam                7
Name: author, dtype: int64

### I think this is the post author tagging `@uofpenn` given all the comments are blank

- as in, the univ hasn't actually commented & replyed...

In [125]:
etl_df[etl_df.author == 'uofpenn']

,url,author,comment
0,https://www.instagram.com/p/CCj83gJhhA6,uofpenn,
0,https://www.instagram.com/p/CDFqYo7BxDv,uofpenn,
0,https://www.instagram.com/p/CDh-J8WhPUT,uofpenn,
0,https://www.instagram.com/p/CCCYYJzBolJ,uofpenn,
0,https://www.instagram.com/p/CEKl7YoBrwL,uofpenn,
0,https://www.instagram.com/p/CDm-ee-BzKl,uofpenn,
0,https://www.instagram.com/p/CDpS3mvhCjA,uofpenn,
0,https://www.instagram.com/p/CCM5tgGhlUT,uofpenn,
0,https://www.instagram.com/p/CDkLXzmB9Ao,uofpenn,
0,https://www.instagram.com/p/CC6TVaThmNo,uofpenn,


### Looking at the text:

In [129]:
comments = []

In [130]:
for com in etl_df.comment:
    comments.append(com)

In [135]:
words_from_comments = []

for comment in comments:
    for word in comment.split():
        words_from_comments.append(word)

## Here is a really quick value counts of all words across comments...

- this is of course with simple `.str.lower()` but really without cleanup or tokenization... 

- lots of ##w given the comments are from a while ago, just goes to show that the instagram webscraping process requires much hand tuning.

- we can start to find relevant hashtags of interest

In [143]:
pd.DataFrame.from_dict({'words':words_from_comments}).words.str.lower().value_counts().head(40)

reply                        709
@uofpenn                     475
.                            331
like                         238
1                            215
likes                        214
to                           129
35w                          125
the                          124
and                           97
30w                           95
this                          87
29w                           85
31w                           85
33w                           85
32w                           84
a                             77
i                             77
34w                           73
is                            72
of                            68
2                             63
that                          58
replies                       56
#blackstudentsspeak           54
#upenn                        54
#penn                         54
#blackivystories              52
you                           51
#universityofpenn             50
#blackstud

### Looking at hashtags:

In [146]:
pd.DataFrame.from_dict({'words':words_from_comments}).words.str.startswith('#').value_counts().head(40)

False    7645
True      562
Name: words, dtype: int64

In [147]:
word_freq_df = pd.DataFrame.from_dict({'words':words_from_comments})

In [148]:
word_freq_df[word_freq_df.words.str.startswith('#')].value_counts()

words                     
#upenn                        54
#blackstudentsspeak           54
#penn                         54
#blackivystories              52
#universityofpenn             50
#blackstudentsmatter          50
#uofpenn                      49
#ivyleague                    49
#blackatpenn                  46
#blackivyleague               45
#universityofpennslyvania     37
#universityofpennsylvania     14
#uofpennsylvania               3
#universityofpennyslvania      2
#universityofpennysylvania     1
#blackpenn                     1
#wtf                           1
dtype: int64

### At this point the comment getter does work nicely in a notebook... 
- I am thinking of revisiting the docker deploymnet now to run for all 600~ images to get all comments